# **RANDOM FOREST**

Desarrollaremos diferentes modelos de random forest variando los parámetros y así obtener el modelo con mejor rendimiento.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

In [3]:
train = pd.read_csv("Dataset train.csv")
train.head()

,NUMPERIODO,nro_documento_hash,nro_telefono_hash,FECACTIVACIONCONTRATO_INT,FECINGRESOCLIENTE_INT,tipo1,tipo2,VCHMESADENDA,VCHPENALIDAD,alto,...,trafico_app_8,trafico_app_9,trafico_total,GIRO,SUBGIRO,TIENE_PROD_1,TIENE_PROD_2,TIENE_PROD_3,TARGET,PREV_TARGET
0,202201,5f58359a37ac456d9050c75aa36720b73063bf4b77f483...,9be57e3928a78ae85be270575f97e82a6bdbd59fc87281...,1072320.0,902100.0,1,0,0.0,0.00,1.0,...,0.000000,0.0,16606.140346,3e33e191fc2f1513037662f262867a04e278e8a0589a32...,8e0b81ed56b34766254d0492d32fa9d5d08a33690db2b6...,1,0,0,0.0,0.0
1,202201,ea411834dcd05b9e5eb4e1bed1a6b99250b33a4293fe46...,119dd6dff66e259a3996de7d885e2c54e8ed5874239346...,1072680.0,781560.0,1,0,7.6,365.57,1.0,...,180.536775,0.0,3705.222886,a9d0bcb92394bd06ed0e020135054d2dbf3e512fdf0b7f...,e159ded5abcf126e18b05856319e4a4ee58a3267f298fd...,1,0,0,0.0,0.0
2,202201,ea411834dcd05b9e5eb4e1bed1a6b99250b33a4293fe46...,73cec44593f362cccdb45773197a0bf2136978cb241c91...,1072680.0,781560.0,1,0,0.0,0.00,0.0,...,980.521564,0.0,4617.043791,a9d0bcb92394bd06ed0e020135054d2dbf3e512fdf0b7f...,e159ded5abcf126e18b05856319e4a4ee58a3267f298fd...,1,0,0,0.0,0.0
3,202201,ea411834dcd05b9e5eb4e1bed1a6b99250b33a4293fe46...,db86e518af696d60b247466fd1bb5823cf81a9442c16b2...,1072680.0,781560.0,0,1,0.0,0.00,0.0,...,0.000000,0.0,0.000000,a9d0bcb92394bd06ed0e020135054d2dbf3e512fdf0b7f...,e159ded5abcf126e18b05856319e4a4ee58a3267f298fd...,1,0,0,0.0,0.0
4,202201,ea411834dcd05b9e5eb4e1bed1a6b99250b33a4293fe46...,90d5ab1ad28da36b0fa821759b0a9df2b248dc92a557a5...,1072680.0,781560.0,1,0,0.0,0.00,1.0,...,37.027894,0.0,450.053533,a9d0bcb92394bd06ed0e020135054d2dbf3e512fdf0b7f...,e159ded5abcf126e18b05856319e4a4ee58a3267f298fd...,1,0,0,0.0,0.0


In [4]:
test = pd.read_csv("Dataset test.csv")
test.drop("TARGET", axis = 1, inplace = True)
test.head()

,NUMPERIODO,nro_documento_hash,nro_telefono_hash,FECACTIVACIONCONTRATO_INT,FECINGRESOCLIENTE_INT,tipo1,tipo2,VCHMESADENDA,VCHPENALIDAD,alto,...,trafico_app_7,trafico_app_8,trafico_app_9,trafico_total,GIRO,SUBGIRO,TIENE_PROD_1,TIENE_PROD_2,TIENE_PROD_3,PREV_TARGET
0,202204,5848c849c6e33ff61bc87f63781cec2cf5a38811446ec1...,ac8e7810e7d278b75f99a38183357eb28a2cb582f1875a...,1080060.0,1079280.0,1,0,0.0,0.0,0.0,...,0.00000,0.000000,0.0,21631.486330,5b8e595b8cebb006e39a3d289e465a5718996785008cbb...,bd1b475ded03bdb3ab78de2f74782ad855c3c4b085d167...,0,0,0,0.0
1,202204,4623594982b1da7de047b2a10eba7533cc83a1b242167d...,93d2534f78d216261f0e0ce2e4d3e938f1d03fb8a4c766...,1080060.0,993180.0,1,0,0.0,0.0,1.0,...,0.00000,0.000000,0.0,5449.355104,5b8e595b8cebb006e39a3d289e465a5718996785008cbb...,bd1b475ded03bdb3ab78de2f74782ad855c3c4b085d167...,0,0,0,0.0
2,202204,b38db111537bc0ebf7297373c3ba7114b6b74751195607...,0095dd8326c84eb25f2a2f888de98d29201dc9bd918829...,1080120.0,716760.0,0,1,0.0,0.0,1.0,...,0.00000,0.000000,0.0,7566.943466,404a18af35694f85854d87b70591b07c315b4ce5f5c7cf...,6eceace9a0758d06fca1f7c802c8ae57ae21361ae0e6a3...,1,0,0,0.0
3,202204,b996b613a4f6cc9b938241b75a4fbde47bb8606938159b...,255a24c2b9ecb57e21346616932f52c9a12924f7bf4738...,1080120.0,912240.0,1,0,0.0,0.0,0.0,...,0.00000,0.000000,0.0,19392.310549,47a5a3efcbc0c9667600a36756e35afd8ed48df33f7e9d...,6b30ae99766ae521a61dccfbebed9aace3f62e0412aa6f...,1,0,0,0.0
4,202204,d1a5dd4012b5399691b0dfe3b24d4f6a303765e0364cc2...,28257d69713585f4741944029f476de65c9eda832e3063...,1079040.0,1007520.0,0,1,0.0,0.0,0.0,...,2.03855,218.083858,0.0,4611.995497,5b8e595b8cebb006e39a3d289e465a5718996785008cbb...,bd1b475ded03bdb3ab78de2f74782ad855c3c4b085d167...,1,0,0,0.0


Para el entrenamiento del modelo descartaremos las variables de tipo objeto.

In [5]:
cols = train.select_dtypes(exclude = 'object').columns
cols

Index(['NUMPERIODO', 'FECACTIVACIONCONTRATO_INT', 'FECINGRESOCLIENTE_INT',
       'tipo1', 'tipo2', 'VCHMESADENDA', 'VCHPENALIDAD', 'alto', 'bajo',
       'medio', 'muy alto', 'NRO_GRUPOS', 'grupo_2', 'grupo_1', 'grupo_11',
       'grupo_5', 'grupo_9', 'grupo_10', 'grupo_3', 'grupo_7', 'grupo_4',
       'grupo_8', 'grupo_6', 'MINUTOS1', 'GIGAS1', 'MENSAJES1', 'MINUTOS2',
       'GIGAS2', 'MENSAJES2', 'MINUTOS3', 'GIGAS3', 'MENSAJES3',
       'MINUTOS_TOTAL', 'MENSAJES_TOTAL', 'GIGAS_TOTAL', 'SMARTPHONE',
       'gamma_1', 'gamma_2', 'gamma_3', 'gamma_4', 'gamma_5', 'gamma_6',
       'gamma_7', 'LANZAMIENTO_INT', 'mins_flujo_1', 'mins_flujo_2',
       'trafico_app_1', 'trafico_app_2', 'trafico_app_3', 'trafico_app_4',
       'trafico_app_5', 'trafico_app_6', 'trafico_app_7', 'trafico_app_8',
       'trafico_app_9', 'trafico_total', 'TIENE_PROD_1', 'TIENE_PROD_2',
       'TIENE_PROD_3', 'TARGET', 'PREV_TARGET'],
      dtype='object')

Realizaremos la división de los datos del dataset de entrenamiento en 3 conjuntos:
* Train set: datos para entrenar los modelos (96%)
* Dev set: datos para comparar los modelos (2%: 11K)
* Test set: datos para probar los modelos (2%: 11K)

Al igual que en el notebook de regresión logística, los datasets de validación y prueba deben reflejar la proporción de registros positivos existentes en el dataset general.

In [6]:
pos_records = train.loc[train['TARGET'] == 1, cols]
pos_records.shape

(16472, 61)

In [7]:
neg_records = train.loc[train['TARGET'] == 0, cols]
neg_records.shape

(531523, 61)

In [8]:
X_pos = pos_records.loc[:, cols != "TARGET"]
Y_pos = pos_records.loc[:, "TARGET"]
X_neg = neg_records.loc[:, cols != "TARGET"]
Y_neg = neg_records.loc[:, "TARGET"]

In [9]:
X_train_pos, X_rem_pos, Y_train_pos, Y_rem_pos = train_test_split(X_pos, Y_pos, test_size = 0.04, shuffle = True, random_state = 0)
X_dev_pos, X_test_pos, Y_dev_pos, Y_test_pos = train_test_split(X_rem_pos, Y_rem_pos, test_size = 0.5, shuffle = True, random_state = 0)

X_train_neg, X_rem_neg, Y_train_neg, Y_rem_neg = train_test_split(X_neg, Y_neg, test_size = 0.04, shuffle = True, random_state = 0)
X_dev_neg, X_test_neg, Y_dev_neg, Y_test_neg = train_test_split(X_rem_neg, Y_rem_neg, test_size = 0.5, shuffle = True, random_state = 0)

X_train = pd.concat([X_train_pos, X_train_neg], axis = 0)
Y_train = pd.concat([Y_train_pos, Y_train_neg], axis = 0)
X_dev = pd.concat([X_dev_pos, X_dev_neg], axis = 0)
Y_dev = pd.concat([Y_dev_pos, Y_dev_neg], axis = 0)
X_test = pd.concat([X_test_pos, X_test_neg], axis = 0)
Y_test = pd.concat([Y_test_pos, Y_test_neg], axis = 0)

In [10]:
print("X_train size: {}".format(X_train.shape))
print("Y_train size: {}".format(Y_train.shape))
print("X_dev size: {}".format(X_dev.shape))
print("Y_dev size: {}".format(Y_dev.shape))
print("X_test size: {}".format(X_test.shape))
print("Y_test size: {}".format(Y_test.shape))

X_train size: (526075, 60)
Y_train size: (526075,)
X_dev size: (10959, 60)
Y_dev size: (10959,)
X_test size: (10961, 60)
Y_test size: (10961,)


In [11]:
print("Proportion train set: {:.3f} %".format(Y_train.value_counts()[1]/Y_train.shape[0] * 100))
print("Proportion dev set: {:.3f} %".format(Y_dev.value_counts()[1]/Y_dev.shape[0] * 100))
print("Proportion test set: {:.3f} %".format(Y_test.value_counts()[1]/Y_test.shape[0] * 100))

Proportion train set: 3.006 %
Proportion dev set: 3.002 %
Proportion test set: 3.011 %


Primero iteraremos sobre el criterio de separación, la profundidad máxima de los árboles y el número mínimo de ejemplos para dividir utilizando un número pequeño de árboles por cada conjunto de parámetros. 

In [36]:
criterion = ['gini', 'entropy']
max_depth = np.arange(5,15)
min_samples_split = np.arange(10, 51, 10)

print("Criterios de separación: {}".format(criterion))
print("Máxima profundidad: {}".format(max_depth))
print("Mínimo nro de ejemplos para dividir: {}".format(min_samples_split))

Criterios de separación: ['gini', 'entropy']
Máxima profundidad: [ 5  6  7  8  9 10 11 12 13 14]
Mínimo nro de ejemplos para dividir: [10 20 30 40 50]


In [37]:
model = Pipeline([('scaler', MinMaxScaler()), ('tree', RandomForestClassifier(n_estimators = 20, random_state = 0))])
results = []

for crit in criterion:
    model.set_params(tree__criterion = crit)

    for depth in max_depth:
        model.set_params(tree__max_depth = depth)

        for samples in min_samples_split:
            model.set_params(tree__min_samples_split = samples)

            model.fit(X_train, Y_train)

            y_train_preds = model.predict_proba(X_train)
            auc_score_train = roc_auc_score(Y_train, y_train_preds[:,1])
            
            y_dev_preds = model.predict_proba(X_dev)
            auc_score_dev = roc_auc_score(Y_dev, y_dev_preds[:,1])

            if (depth in [9, 14]) and samples == 50:
                print("Criterio: {}".format(crit))
                print("Depth: {}".format(depth))
                print("Min_samples_split {}".format(samples))
                print("Train score: {}".format(auc_score_train))
                print("Dev score: {}".format(auc_score_dev))

            results.append([crit, depth, samples, auc_score_train, auc_score_dev])


Criterio: gini
Depth: 9
Min_samples_split 50
Train score: 0.7914479620694883
Dev score: 0.7524612054545402
Criterio: gini
Depth: 14
Min_samples_split 50
Train score: 0.8694147268467599
Dev score: 0.7629712604402863
Criterio: entropy
Depth: 9
Min_samples_split 50
Train score: 0.7971288740205267
Dev score: 0.7610074429483569
Criterio: entropy
Depth: 14
Min_samples_split 50
Train score: 0.8877736978935159
Dev score: 0.7725188504176115


In [49]:
results_df = pd.DataFrame(np.array(results), columns = ['criterio', 'max_depth', 'min_samples_split', 'Train score', 'Dev score'])
results_df['Train score'] = results_df['Train score'].astype(np.float32)
results_df['Dev score'] = results_df['Dev score'].astype(np.float32)

results_df['Variance'] = results_df['Train score'] - results_df['Dev score']

results_df.sort_values(by = ['Dev score', 'Train score'], ascending = False).head(10)

,criterio,max_depth,min_samples_split,Train score,Dev score,Variance
93,entropy,13,40,0.870913,0.782567,0.088346
98,entropy,14,40,0.891023,0.779887,0.111136
47,gini,14,30,0.878706,0.779347,0.099358
91,entropy,13,20,0.878193,0.778591,0.099602
97,entropy,14,30,0.889706,0.777775,0.111931
94,entropy,13,50,0.871451,0.777160,0.094291
43,gini,13,40,0.856639,0.776997,0.079642
28,gini,10,40,0.807824,0.776425,0.031399
86,entropy,12,20,0.857182,0.775960,0.081222
90,entropy,13,10,0.881130,0.775375,0.105755


* Vemos que a medida que aumenta la máxima profundidad de los árboles el score en el conjunto de entrenamiento aumenta en mayor medida que el score del conjunto de validación.
* La diferencia mínima entre los scores se obtiene para una máxima profundidad de 10, mientras que el mayor score del conjunto de validación se obtiene para una máxima profundidad de 13.
* A pesar que el número mínimo de ejemplos para dividir varía entre 10 y 50, el score de los modelos apenas varía.

Iteraremos nuevamente sobre la máxima profundidad entre 10 y 14, los dos criterios utilizados anteriormente, y dos valores para el mínimo número de ejemplos para dividir (30 y 40, que son los valores que más se repiten).

In [51]:
criterion = ['gini', 'entropy']
max_depth = np.arange(10,15)
min_samples_split = np.arange(30, 41, 10)

print("Criterios de separación: {}".format(criterion))
print("Máxima profundidad: {}".format(max_depth))
print("Mínimo nro de ejemplos para dividir: {}".format(min_samples_split))

Criterios de separación: ['gini', 'entropy']
Máxima profundidad: [10 11 12 13 14]
Mínimo nro de ejemplos para dividir: [30 40]


In [54]:
model2 = Pipeline([('scaler', MinMaxScaler()), ('tree', RandomForestClassifier(n_estimators = 128, random_state = 0))])
results2 = []

for crit in criterion:
    model2.set_params(tree__criterion = crit)

    for depth in max_depth:
        model2.set_params(tree__max_depth = depth)

        for samples in min_samples_split:
            model2.set_params(tree__min_samples_split = samples)

            model2.fit(X_train, Y_train)

            y_train_preds = model2.predict_proba(X_train)
            auc_score_train = roc_auc_score(Y_train, y_train_preds[:,1])
            
            y_dev_preds = model2.predict_proba(X_dev)
            auc_score_dev = roc_auc_score(Y_dev, y_dev_preds[:,1])

            if samples == 40:
                print("Criterio: {}".format(crit))
                print("Depth: {}".format(depth))
                print("Min_samples_split {}".format(samples))
                print("Train score: {}".format(auc_score_train))
                print("Dev score: {}".format(auc_score_dev))

            results2.append([crit, depth, samples, auc_score_train, auc_score_dev])


Criterio: gini
Depth: 10
Min_samples_split 40
Train score: 0.8225892595521604
Dev score: 0.7716674720567758
Criterio: gini
Depth: 11
Min_samples_split 40
Train score: 0.8443825103189425
Dev score: 0.7754306930834622
Criterio: gini
Depth: 12
Min_samples_split 40
Train score: 0.8617898491293857
Dev score: 0.7757938334758254
Criterio: gini
Depth: 13
Min_samples_split 40
Train score: 0.8834968221561095
Dev score: 0.7833951911062057
Criterio: gini
Depth: 14
Min_samples_split 40
Train score: 0.9023487628894639
Dev score: 0.7775720776491377
Criterio: entropy
Depth: 10
Min_samples_split 40
Train score: 0.8310820707824482
Dev score: 0.7659923311611628
Criterio: entropy
Depth: 11
Min_samples_split 40
Train score: 0.8520551957388897
Dev score: 0.7733762334621004
Criterio: entropy
Depth: 12
Min_samples_split 40
Train score: 0.8752822161744179
Dev score: 0.7758571685915012
Criterio: entropy
Depth: 13
Min_samples_split 40
Train score: 0.8961378538748299
Dev score: 0.7808003099560514
Criterio: entrop

In [56]:
results2_df = pd.DataFrame(np.array(results2), columns = ['criterio', 'max_depth', 'min_samples_split', 'Train score', 'Dev score'])
results2_df['Train score'] = results2_df['Train score'].astype(np.float32)
results2_df['Dev score'] = results2_df['Dev score'].astype(np.float32)

results2_df['Variance'] = results2_df['Train score'] - results2_df['Dev score']

results2_df.sort_values(by = ['Dev score', 'Train score'], ascending = False).head()

,criterio,max_depth,min_samples_split,Train score,Dev score,Variance
7,gini,13,40,0.883497,0.783395,0.100102
19,entropy,14,40,0.915711,0.782321,0.133391
8,gini,14,30,0.905556,0.781799,0.123757
17,entropy,13,40,0.896138,0.780800,0.115338
16,entropy,13,30,0.900424,0.780344,0.120080


Calcularemos el score en el set de prueba para los mejores 5 modelos encontrados.

In [59]:
results2_df['max_depth'] = results2_df['max_depth'].astype(int)
results2_df['min_samples_split'] = results2_df['min_samples_split'].astype(int)

results2_df.dtypes

criterio              object
max_depth              int32
min_samples_split      int32
Train score          float32
Dev score            float32
Variance             float32
dtype: object

In [73]:
top5 = results2_df.sort_values(by = ['Dev score', 'Train score'], ascending = False).head()
top5 = top5.reset_index()
top5.drop('index', axis = 1, inplace = True)

top5['Test score'] = np.zeros(top5.shape[0])

model2 = Pipeline([('scaler', MinMaxScaler()), ('tree', RandomForestClassifier(n_estimators = 128, random_state = 0))])

for i in range(top5.shape[0]):

    criterio = top5.loc[i, 'criterio']
    depth = top5.loc[i, 'max_depth']
    min_samples_split = top5.loc[i, 'min_samples_split']

    model2.set_params(tree__criterion = criterio, tree__max_depth = depth, tree__min_samples_split = min_samples_split)

    model2.fit(X_train, Y_train)

    y_preds = model2.predict_proba(X_test)
    test_score = roc_auc_score(Y_test, y_preds[:,1])

    top5.loc[i, 'Test score'] = test_score

    print("Model {} score: {}".format(i, test_score))


Model 0 score: 0.7613795275680328
Model 1 score: 0.7665808684151267
Model 2 score: 0.761118284719075
Model 3 score: 0.76826376833902
Model 4 score: 0.7696171573699557


In [76]:
top5['Dev - Test'] = top5['Dev score'] - top5['Test score']

top5.sort_values(by = ['Test score'], ascending = False)

,criterio,max_depth,min_samples_split,Train score,Dev score,Variance,Test score,Dev - Test
4,entropy,13,30,0.900424,0.780344,0.120080,0.769617,0.010727
3,entropy,13,40,0.896138,0.780800,0.115338,0.768264,0.012537
1,entropy,14,40,0.915711,0.782321,0.133391,0.766581,0.015740
0,gini,13,40,0.883497,0.783395,0.100102,0.761380,0.022016
2,gini,14,30,0.905556,0.781799,0.123757,0.761118,0.020681


Nos quedaremos con el modelo 4, ya que presenta el mayor valor de test score y la menor diferencia entre el score del conjunto de validación y el de prueba.

In [12]:
tree_model = RandomForestClassifier(criterion = 'entropy', max_depth = 13, min_samples_split = 30, n_estimators = 128, random_state = 0)
final_model = Pipeline([('scaler', MinMaxScaler()), ('tree', tree_model)])

final_model.fit(X_train, Y_train)

y_train_preds = final_model.predict_proba(X_train)
auc_score_train = roc_auc_score(Y_train, y_train_preds[:,1])

y_dev_preds = final_model.predict_proba(X_dev)
auc_score_dev = roc_auc_score(Y_dev, y_dev_preds[:,1])

y_test_preds = final_model.predict_proba(X_test)
auc_score_test = roc_auc_score(Y_test, y_test_preds[:,1])

print("Train AUC score: {}".format(auc_score_train))
print("Dev AUC score: {}".format(auc_score_dev))
print("Test AUC score: {}".format(auc_score_test))

Train AUC score: 0.9004238679285508
Dev AUC score: 0.7803438110297461
Test AUC score: 0.7696171573699557


Procedemos a guardar el modelo, para ello usaremos la librería *joblib*.


In [13]:
import joblib

joblib.dump(final_model, 'random_forest_model.pkl')

['random_forest_model.pkl']

Por último, realizaremos las predicciones para el mes de abril.

In [16]:
cols_test = test.select_dtypes(exclude = 'object').columns
cols_test

Index(['NUMPERIODO', 'FECACTIVACIONCONTRATO_INT', 'FECINGRESOCLIENTE_INT',
       'tipo1', 'tipo2', 'VCHMESADENDA', 'VCHPENALIDAD', 'alto', 'bajo',
       'medio', 'muy alto', 'NRO_GRUPOS', 'grupo_2', 'grupo_1', 'grupo_11',
       'grupo_5', 'grupo_9', 'grupo_10', 'grupo_3', 'grupo_7', 'grupo_4',
       'grupo_8', 'grupo_6', 'MINUTOS1', 'GIGAS1', 'MENSAJES1', 'MINUTOS2',
       'GIGAS2', 'MENSAJES2', 'MINUTOS3', 'GIGAS3', 'MENSAJES3',
       'MINUTOS_TOTAL', 'MENSAJES_TOTAL', 'GIGAS_TOTAL', 'SMARTPHONE',
       'gamma_1', 'gamma_2', 'gamma_3', 'gamma_4', 'gamma_5', 'gamma_6',
       'gamma_7', 'LANZAMIENTO_INT', 'mins_flujo_1', 'mins_flujo_2',
       'trafico_app_1', 'trafico_app_2', 'trafico_app_3', 'trafico_app_4',
       'trafico_app_5', 'trafico_app_6', 'trafico_app_7', 'trafico_app_8',
       'trafico_app_9', 'trafico_total', 'TIENE_PROD_1', 'TIENE_PROD_2',
       'TIENE_PROD_3', 'PREV_TARGET'],
      dtype='object')

In [17]:
target_abril = final_model.predict_proba(test.loc[:, cols_test])
target_abril

array([[0.9644726 , 0.0355274 ],
       [0.96386334, 0.03613666],
       [0.98143765, 0.01856235],
       ...,
       [0.97615999, 0.02384001],
       [0.97424836, 0.02575164],
       [0.96445971, 0.03554029]])

In [18]:
import warnings
warnings.filterwarnings("ignore")

In [19]:
predictions = test[['nro_telefono_hash']]
predictions['TARGET'] = target_abril[:, 1]
predictions.head()

,nro_telefono_hash,TARGET
0,ac8e7810e7d278b75f99a38183357eb28a2cb582f1875a...,0.035527
1,93d2534f78d216261f0e0ce2e4d3e938f1d03fb8a4c766...,0.036137
2,0095dd8326c84eb25f2a2f888de98d29201dc9bd918829...,0.018562
3,255a24c2b9ecb57e21346616932f52c9a12924f7bf4738...,0.031756
4,28257d69713585f4741944029f476de65c9eda832e3063...,0.016664


In [20]:
predictions.to_csv('random_forest_predictions.csv', index = False)